## CNN for Run out Prediction

In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers

base_dir = 'dataset'  

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
    rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 165 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


## Model initialization and compiling 

In [2]:

# Model creation
model = models.Sequential([
  layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


c:\Users\Hanzalah\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


c:\Users\Hanzalah\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


## Validation accuracy and Validation Loss

In [3]:

validation_results = model.evaluate(val_generator)

print("Validation loss:", validation_results[0])
print("Validation accuracy:", validation_results[1])

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.6314 - loss: 0.7724
Validation loss: 0.7577418088912964
Validation accuracy: 0.6346153616905212


## Saving model 

In [4]:
model.save("model.h5")

## Testing on images 

In [ ]:
## optional 
import numpy as np
from tensorflow.keras.preprocessing import image
img_path = 'out.jpg'  
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0
prediction = model.predict(img_array)
predicted_class = int(round(prediction[0][0]))
threshold = 0.5
predicted_class = 1 if prediction[0][0] > threshold else 0
class_labels = {0: 'not out', 1: 'out'}
predicted_label = class_labels[predicted_class]
print("Predicted Label:", predicted_label)